In [2]:
from time import time

import numpy as np
import pandas as pd
import csv

from mlxtend.preprocessing import TransactionEncoder

In [3]:
# Function to determine whether an item/itemset is contained in a list/basket
def isInList(aList, item):
    if item in aList:
        return 1
    else:
        return -1   

    
# Function to remove strings (e.g., white space) from the list items
def removeEmptyValues(aList, symbolToRemove):
    while(True):
        val = isInList(aList, symbolToRemove)
        if(val == 1):
            aList.remove(symbolToRemove)    
        else:
            break

        

# Function to create a list of lists from a CSV file containing transaction data
def generateListOfListsFromCSV(fullFileName):

    with open(fullFileName, newline='', encoding='utf-8-sig') as f:
        reader = csv.reader(f, delimiter=",")
        data = list(reader)

    listOfLists = []

    for i in range(len(data)):
        removeEmptyValues(data[i], '')
        listOfLists.append(data[i])
        
    return listOfLists

 

# Function to create a one-hot encoded DataFrame object from the Python list of lists        
def oneHotEncodedDataFrame(data):

    # Transform the transaction data into a one-hot encoded NumPy boolean array 
    te = TransactionEncoder()
    te_ary = te.fit(data).transform(data)


    # Read the one-hot encoded data as a Pandas DataFrame object
    dataFrame = pd.DataFrame(te_ary, columns=te.columns_)
    
    return dataFrame

Itemsets = []

def compare_set_except_last_item(s1, s2):
    if isinstance(s1, str):
        s1 = [s1]
    if isinstance(s2, str):
        s2 = [s2]
    assert len(s1) == len(s2)
    for i in range(len(s1)-1):
        if s1[i] != s2[i]:
            return False
    return True

# Function to create a list of items & baskets from a one-hot encoded DataFrame object
def generateItemsBaskets(dataFrame):    
    # Create a list of items from the DataFrame object
    items = []
    for col in dataFrame.columns: 
        items.append(col)
        
    # Create a list of baskets from the DataFrame object
    baskets = []
    transaction_id = 1

    for i in range(len(dataFrame)):
        itemset = []
        for j in range(len(items)):
            if(dataFrame.iloc[i][j] == True):
                itemset.append(items[j])
        baskets.append({transaction_id: itemset})
        transaction_id += 1
        
    return items, baskets

def apriori_gen(itemsets: Itemsets) -> Itemsets:
    if len(itemsets) <= 1: return itemsets
    
    try:
        if isinstance(itemsets[0], str):
            itemsets = [[item] for item in itemsets]
    except:
        pass
    # pad null for each itemset. cover case where each itemset has len of 1
    #if len(itemsets[0]) == 1:
    #    padded_sets = list(map(lambda x: [None, *x], itemsets))
    #    itemsets = padded_sets
    i, j = 0, 1

    result_sets = []
    while (i < len(itemsets)):
        while (j < len(itemsets) and compare_set_except_last_item(itemsets[i], itemsets[j])):
            result_sets.append(sorted(list(set().union(itemsets[i], itemsets[j]))))
            j += 1
        i += 1
        j = i + 1
    return result_sets

def convert_basket_2onehot(basket):
    
    if isinstance(basket, str):
        basket = [basket]
    
    one_hot = []
    for item in range(len(basket)):
        one_hot.append(int(basket[item][1:]))
    
    translated = []
    for i in range(100):
        if i in one_hot:
            translated.append(True)
        else:
            translated.append(False)
    return [one_hot,translated] #translated

def check2lists(list1,list2,length):
    ## list1 is 2-d dataframe.values
    ## list2 is 1-d 
    counts = []
    count = 0
    for list_n in list1:
        if len(list_n) != len(list2):
            return 'Yo! lists are WRONG'
        for elem in range(len(list_n)):
            if list_n[elem] == True and list2[elem] == True:
                    count += 1 # np.floor_divide(count / length)
    
        counts.append(np.floor_divide(count,length))    
        count = 0
    return counts

In [4]:
## This function works for Question 1 (only outputting frequent-3 itemset) AND
#### also works for frequent-k itemsets (for Question 10)
def aprioriFrequentItemsets(df, min_support=1, max_len=None):
    
    ## step1: create all frequent-1 possibilities
    items = list(df.columns) # !!change to df
    
    report_only_frequent3 = False
    ## if no max_len is set, this function will work through frequent-1,2,3 itemsets
    if max_len == None:
        report_only_frequent3 = True
        max_len = 3
    

    for j in range(1, max_len+1):
        C_n = {}
        support_pct = {}
        
        if j > max_len:
            break

        if j == 1:
            for item in items:
                ## np.sum() works for frequent-1 itemsets
                C_n[item] = np.sum(df[item]) ## !!change to df
                support_pct[item] = C_n[item] / len(items)

        else:

            ## apriori_gen new combinations from current combinations
            new_items = apriori_gen(F_keys)
            
            # count occurances of each basket
            for basket in range(len(new_items)):

                # select current basket
                basket_n = tuple(new_items[basket])


                # convert current basket to one_hot
                curr_basket = convert_basket_2onehot(new_items[basket])[1]

                check_baskets = check2lists(dataset_onehot.values,curr_basket,j)
                num_baskets = np.sum(check_baskets)
                
                # store the number of times this particular basket combination occurred 
                C_n[basket_n] = num_baskets
                # calculate support
                support_pct[basket_n] = num_baskets / len(new_items)
    
                ## ensure candidates meet the min_support    
        F_n = {}
        for key in C_n.keys():
            if C_n[key] < min_support:
                del support_pct[key]
            else:
                F_n[key] = C_n[key]
                
        F_keys = list(F_n.keys())
        
        ## ensure baskets are list of lists
        try:
            if isinstance(F_keys[0], str):
                F_keys = [[key] for key in F_keys]
        except:
            # F_keys[0] not valid
            # debugging
            pass
        ## if max_len == 1, return dataframe of frequent-1 itemset counts and support
        if max_len < 2:
            print('--Results--')
            print('Frequent-'+str(j)+' Itemset:\n')
            return pd.DataFrame([F_n,support_pct],index=['support','support_percent'])  ## [F_n, support])
        
        if report_only_frequent3:
            if j < 3:
                continue
            else:
                # need to figure out how we want to display support and basket
                basket_support = pd.DataFrame([F_n,support_pct],index=['support','support_percent'])
                print('--Results--')
                print('Frequent-'+str(j)+' Itemset:\n')
                print(basket_support)
                print()
            
        else:
            # need to figure out how we want to display support and basket
            basket_support = pd.DataFrame([F_n,support_pct],index=['support','support_percent'])
            print('--Results--')
            print('Frequent-'+str(j)+' Itemset:\n')
            print(basket_support)
            print()
    

    return
    
    ## additional notes
    # apriori_gen() always performed on the 'columns' of current reported, OR
    ## keys of dictionaries -- each will be a list of lists as input
    ## should be F_keys
    

In [7]:
dataset = generateListOfListsFromCSV('transactionlarge.csv')

print("\nDataset (list of lists):")
for i in range(len(dataset)):
    print(dataset[i])

# Convert the **list of lists** transaction data as one-hot encoded Pandas DataFrame object.
dataset_onehot = oneHotEncodedDataFrame(dataset)

print("\nOne-hot Encoded Dataset:")
print(dataset_onehot)


Dataset (list of lists):
['M52', 'M42', 'M34', 'M64', 'M81', 'M43', 'M26', 'M59', 'M16']
['M77', 'M14', 'M15', 'M21', 'M97', 'M82', 'M12']
['M18', 'M93', 'M51', 'M36', 'M49']
['M77', 'M56', 'M14', 'M24', 'M3', 'M95']
['M93', 'M0', 'M95', 'M1', 'M27', 'M2', 'M17']
['M33', 'M6', 'M58', 'M21', 'M89']
['M25', 'M99', 'M72', 'M10', 'M47']
['M0', 'M6', 'M1', 'M21', 'M57', 'M47']
['M74', 'M55', 'M23', 'M28', 'M80', 'M66']
['M98', 'M22', 'M6', 'M13', 'M42', 'M83', 'M92', 'M79', 'M71', 'M84']
['M18', 'M33', 'M4', 'M7', 'M85', 'M34', 'M30', 'M64', 'M59']
['M33', 'M51', 'M24', 'M11', 'M95', 'M79', 'M97', 'M54', 'M10', 'M87']
['M77', 'M18', 'M98', 'M75', 'M70', 'M36', 'M38', 'M57', 'M60']
['M45', 'M35', 'M8', 'M57', 'M80', 'M66']
['M75', 'M14', 'M46', 'M59', 'M10']
['M77', 'M98', 'M75', 'M35', 'M73', 'M22', 'M46', 'M79', 'M29', 'M71', 'M2', 'M81']
['M77', 'M98', 'M51', 'M35', 'M73', 'M79', 'M29', 'M2', 'M30', 'M10']
['M25', 'M45', 'M70', 'M8', 'M41', 'M65', 'M5', 'M15', 'M34', 'M63', 'M96']
['M18'

In [102]:
%%time
# Question 2 & 3
aprioriFrequentItemsets(dataset_onehot,3)

--Results--
Frequent-3 Itemset:

                 (M0, M16, M20)  (M0, M16, M46)  (M0, M16, M51)  \
support                3.000000        4.000000        5.000000   
support_percent        0.000117        0.000156        0.000195   

                 (M0, M16, M73)  (M0, M20, M46)  (M0, M20, M51)  \
support                3.000000        3.000000        3.000000   
support_percent        0.000117        0.000117        0.000117   

                 (M0, M31, M47)  (M0, M42, M47)  (M0, M42, M70)  \
support                4.000000        5.000000        5.000000   
support_percent        0.000156        0.000195        0.000195   

                 (M0, M42, M72)  ...  (M71, M8, M89)  (M71, M83, M88)  \
support                6.000000  ...        5.000000         6.000000   
support_percent        0.000234  ...        0.000195         0.000234   

                 (M72, M78, M95)  (M72, M83, M88)  (M75, M77, M98)  \
support                 4.000000         3.000000         5.000000   
s

In [103]:
%%time
## Q4-minimum support count = 4
aprioriFrequentItemsets(dataset_onehot,4)

--Results--
Frequent-3 Itemset:

                 (M0, M16, M46)  (M0, M16, M51)  (M0, M31, M47)  \
support                4.000000        5.000000        4.000000   
support_percent        0.000426        0.000532        0.000426   

                 (M0, M42, M47)  (M0, M42, M70)  (M0, M42, M72)  \
support                5.000000        5.000000        6.000000   
support_percent        0.000532        0.000532        0.000639   

                 (M0, M46, M51)  (M0, M47, M70)  (M0, M47, M72)  \
support                4.000000        5.000000        5.000000   
support_percent        0.000426        0.000532        0.000532   

                 (M0, M70, M72)  ...  (M68, M7, M78)  (M69, M7, M99)  \
support                5.000000  ...        4.000000        5.000000   
support_percent        0.000532  ...        0.000426        0.000532   

                 (M69, M74, M99)  (M7, M76, M91)  (M7, M85, M95)  \
support                 4.000000        4.000000        4.000000   
support_

In [104]:
%%time
## Q4-minimum support count = 5
aprioriFrequentItemsets(dataset_onehot,5)

--Results--
Frequent-3 Itemset:

                 (M0, M16, M51)  (M0, M42, M47)  (M0, M42, M70)  \
support                5.000000        5.000000        5.000000   
support_percent        0.001683        0.001683        0.001683   

                 (M0, M42, M72)  (M0, M47, M70)  (M0, M47, M72)  \
support                 6.00000        5.000000        5.000000   
support_percent         0.00202        0.001683        0.001683   

                 (M0, M70, M72)  (M1, M12, M15)  (M1, M12, M41)  \
support                5.000000        5.000000        5.000000   
support_percent        0.001683        0.001683        0.001683   

                 (M1, M12, M88)  ...  (M62, M71, M89)  (M62, M8, M89)  \
support                5.000000  ...         5.000000        5.000000   
support_percent        0.001683  ...         0.001683        0.001683   

                 (M63, M65, M86)  (M63, M65, M92)  (M63, M86, M92)  \
support                 5.000000         7.000000         5.000000   
s

In [107]:
%%time
## proof of Q10 w/ same function
aprioriFrequentItemsets(dataset_onehot,3,3)

--Results--
Frequent-1 Itemset:

                    M0     M1    M10    M11    M12    M13   M14   M15    M16  \
support          22.00  28.00  35.00  28.00  23.00  18.00  30.0  40.0  19.00   
support_percent   0.22   0.28   0.35   0.28   0.23   0.18   0.3   0.4   0.19   

                   M17  ...    M90    M91    M92    M93    M94    M95    M96  \
support          27.00  ...  22.00  34.00  31.00  32.00  21.00  25.00  15.00   
support_percent   0.27  ...   0.22   0.34   0.31   0.32   0.21   0.25   0.15   

                   M97    M98    M99  
support          19.00  31.00  24.00  
support_percent   0.19   0.31   0.24  

[2 rows x 99 columns]

--Results--
Frequent-2 Itemset:

                 (M0, M1)  (M0, M12)  (M0, M14)  (M0, M16)  (M0, M20)  \
support          3.000000   3.000000   4.000000   5.000000   5.000000   
support_percent  0.000618   0.000618   0.000825   0.001031   0.001031   

                 (M0, M31)  (M0, M32)  (M0, M38)  (M0, M42)  (M0, M43)  ...  \
support     

#### Question 5 - 8

In [13]:
def bruteForceFrequentItemsets(dataFrame, min_support=3, max_len=None):
    
    
    # Create a list of items & baskets from a one-hot encoded DataFrame object
    items, baskets = generateItemsBaskets(dataFrame)   
    
    # Declare a list to store frequent 3-itemsets
    all_itemsets = []
    freq_itemsets = pd.DataFrame(columns=['Itemsets', 'Support_count', 'Support'])
    
    '''
    Brute-force Aproach:
    
    - Identify candidate 3-itemsets
        -- Go through every consequtive pairs of distinct items the transactions.
    - Identify frequent 3-itemsets:
        -- Determine whether the candidate itemsets exist in the baskets, and increment their basket count.
        -- If the basket count of an itemset is greater than or equal to the minimum support count,
     store it in the frequent itemsets list.
    
    '''
    for i in range(len(items)):

        for j in range(len(items)):
            
            for k in range(len(items)):

                if(k != i and k!=j and j!=i and k > i and k > j  and j > i):
                    basket_count = 0
                    basket_item = 1
                    for b in baskets:
                        val1 = isInList(b.get(basket_item), items[i])
                        val2 = isInList(b.get(basket_item), items[j])
                        val3 = isInList(b.get(basket_item), items[k])
                        if(val1 != -1 and val2 != -1 and val3 != -1):
                            basket_count += 1
                        basket_item += 1


                    if(basket_count >= min_support):
                        all_itemsets.append([items[i], items[j], items[k]])
    
    for itemset in range(len(all_itemsets)):
        support_count = 1
        basket_item = 1
        total_baskets = len(baskets)
        for basket in baskets:
            val1 = isInList(basket.get(basket_item), all_itemsets[itemset][0])
            val2 = isInList(basket.get(basket_item), all_itemsets[itemset][1])
            val3 = isInList(basket.get(basket_item), all_itemsets[itemset][2])
            if(val1 != -1 and val2 != -1 and val3 != -1):
                support_count += 1
            basket_item += 1
            
        if (support_count >= min_support):
            support = support_count / total_baskets
            freq_itemsets = freq_itemsets.append({'Itemsets': all_itemsets[itemset], 'Support_count': support_count, 'Support': support}, ignore_index=True)

    return freq_itemsets

In [14]:
# Convert the **list of lists** transaction data as one-hot encoded Pandas DataFrame object.
dataset_onehotencoded = oneHotEncodedDataFrame(dataset)
  
t0 = time()    

# Generate frequent 3-itemsets using the brute-force function
frequent_itemsets = bruteForceFrequentItemsets(dataset_onehotencoded, min_support=3)

time_brute_force = time() - t0

print("Brute-force: Running Time = %0.3fs" % time_brute_force)
print("\n____________________")
print("Frequent 3-itemsets for min support = 3:")
print(frequent_itemsets)

Brute-force: Running Time = 82.298s

____________________
Frequent 3-itemsets for min support = 3:
            Itemsets Support_count   Support
0     [M0, M23, M27]             4  0.013333
1     [M0, M23, M50]             5  0.016667
2     [M0, M23, M55]             6  0.020000
3     [M0, M23, M75]             4  0.013333
4     [M0, M27, M50]             4  0.013333
..               ...           ...       ...
728   [M74, M8, M95]             5  0.016667
729  [M74, M84, M89]             4  0.013333
730  [M77, M79, M98]             6  0.020000
731  [M87, M91, M93]             4  0.013333
732   [M9, M91, M93]             4  0.013333

[733 rows x 3 columns]


In [15]:
#Question 7
# Convert the **list of lists** transaction data as one-hot encoded Pandas DataFrame object.
dataset_onehotencoded = oneHotEncodedDataFrame(dataset)
  
t0 = time()    

# Generate frequent 3-itemsets using the brute-force function
frequent_itemsets = bruteForceFrequentItemsets(dataset_onehotencoded, min_support=4)

time_brute_force = time() - t0

print("Brute-force: Running Time = %0.3fs" % time_brute_force)
print("\n____________________")
print("Frequent 3-itemsets for min support = 4:")
print(frequent_itemsets)

Brute-force: Running Time = 79.830s

____________________
Frequent 3-itemsets for min support = 4:
            Itemsets Support_count   Support
0     [M0, M23, M50]             5  0.016667
1     [M0, M23, M55]             6  0.020000
2     [M0, M37, M51]             5  0.016667
3     [M0, M47, M51]             6  0.020000
4     [M0, M47, M72]             6  0.020000
..               ...           ...       ...
254  [M71, M76, M99]             5  0.016667
255   [M73, M79, M9]             5  0.016667
256  [M73, M84, M89]             7  0.023333
257   [M74, M8, M95]             5  0.016667
258  [M77, M79, M98]             6  0.020000

[259 rows x 3 columns]


In [18]:
#Question 7
# Convert the **list of lists** transaction data as one-hot encoded Pandas DataFrame object.
dataset_onehotencoded = oneHotEncodedDataFrame(dataset)
  
t0 = time()    

# Generate frequent 3-itemsets using the brute-force function
frequent_itemsets = bruteForceFrequentItemsets(dataset_onehotencoded, min_support=5)

time_brute_force = time() - t0

print("Brute-force: Running Time = %0.3fs" % time_brute_force)
print("\n____________________")
print("Frequent 3-itemsets for min support = 5:")
print(frequent_itemsets)

Brute-force: Running Time = 87.625s

____________________
Frequent 3-itemsets for min support = 5:
            Itemsets Support_count   Support
0     [M0, M23, M55]             6  0.020000
1     [M0, M47, M51]             6  0.020000
2     [M0, M47, M72]             6  0.020000
3     [M0, M47, M74]             7  0.023333
4     [M0, M51, M72]             6  0.020000
..               ...           ...       ...
180  [M66, M68, M92]             8  0.026667
181  [M66, M87, M92]             6  0.020000
182  [M68, M87, M92]             6  0.020000
183  [M73, M84, M89]             7  0.023333
184  [M77, M79, M98]             6  0.020000

[185 rows x 3 columns]
